### There is more confusion on whether grid should be used in machine learning or are we using averages from dataset to setup a model
### Untill that is clear, the ML works stops here

#### The whole function of reading all simulations will be used only when all things are ready,
####  for now it is of no use, so can be saved as raw.

In [ ]:
#-----------steps involves
#work with one piece of data only
#split up the chunk
#setup univariate gaussian 
#fit the gaussian cost function between leafcn and
#plot the raw lines without any hyperparamter change

In [2]:
#read all the data and filter for only some
from ml_utils import *

#### The function is tested with single simulation of 1000 dataset only 

In [348]:
#this is a little different function than 
#what we have added in utils before, this 
#does not read require var to pass into and 
#and we should also not require that\

file='/glade/campaign/cgd/tss/projects/PPE/PPEn11_LHC/transient/hist/PPEn11_transient_LHC0001.clm2.h0.2005-02-01-00000.nc'
latin_hypercube = xr.open_dataset(file)

#bring in single data 
#subset only the TSA, and LeafCN from the dataset

subset_latin_hypercube = latin_hypercube[["LEAFN", "LEAFCN"]]


In [349]:
#create an array for both TSA and Leafcn
tsa = subset_latin_hypercube["LEAFN"].values
leafcn = subset_latin_hypercube["LEAFCN"].values

leafn = subset_latin_hypercube["LEAFN"].values
leafcn = subset_latin_hypercube["LEAFCN"].values


In [361]:
#flatten the list to single column, i tried without flattening that does not make sense
leafn = np.array(leafn).flatten()
leafcn = np.array(leafcn).flatten()


In [362]:
#there should be no missing values becuase machine does not learn anything from
#that and it also does not how to interpret the data
np.unique(leafn)
np.isnan(leafcn).sum()


0

#### The K nearest method imputer makes more sense to me because i can use four nearby grid cells values
#### to impute the nan values

In [363]:
from sklearn.impute import KNNImputer

# Create a KNN imputer with k=4 (number of neighbors), not required for TSA as it no na values
knn_imputer = KNNImputer(n_neighbors=4)
leafcn= knn_imputer.fit_transform(np.array(leafcn).reshape(-1,1))


#### Data is cleaned, and it can be now fit using machine learning. But, this data should be shuffled
#### otherwise there is specific grid and time and model won't perform better. So, do random arrnagement of the 
#### values keeping the relationships of input and output

In [364]:
import numpy as np

# Assuming tsa and leafcn have the same length
num_samples = len(leafn)
indices = np.arange(num_samples)
np.random.shuffle(indices)

# Shuffle tsa and leafcn using the shuffled indices
shuffled_leafn = leafn[indices]
shuffled_leafcn = leafcn[indices]


#### Data preprocessing and selecting only 1000 rows

In [365]:
# split the dataset into xtrain and ytrain 
from sklearn.model_selection import train_test_split

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(shuffled_leafn, 
                                                    shuffled_leafcn, 
                                                    test_size=0.2, 
                                                    random_state=42)

#---------use standard scaler to transform the dataset
from sklearn.preprocessing import StandardScaler
# Assuming tsa and leafcn are one-dimensional lists

# Create StandardScaler objects
train_x_scale= StandardScaler()

# Fit the scalers to the data and transform the data
X_train= train_x_scale.fit_transform(np.array(X_train).reshape(-1, 1))
X_test = train_x_scale.fit_transform(np.array(X_test).reshape(-1, 1))

#for working with ml subsetting only 1000 rows
X_train = X_train[:1000]
y_train = y_train[:1000]


(1000, 1)

#### This is based model with no hyperparameter changing, no cross validation


In [366]:

#two packages that can be used to instantiate the gaussian model
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process.kernels import ConstantKernel, RBF

#instantiate the model and tune nothing in the beginning
kernel = ConstantKernel(constant_value = 3, constant_value_bounds=(1e-2, 1e2)) \
              * RBF(length_scale=1, length_scale_bounds=(1e-2, 1e2))
gp_model = GaussianProcessRegressor(kernel=kernel, random_state=42)

# Fit the model to the training data
gp_model.fit(X_train, y_train)

# Evaluate the model on the testing data
score = gp_model.score(X_test, y_test)
print("Model Score:", score)

/glade/u/home/sbhattarai/.conda/envs/meds-py/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:434: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/glade/u/home/sbhattarai/.conda/envs/meds-py/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:424: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 0.01. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


Model Score: -915203.191497723


#### This is a model with 4 folds cross validation, neaning training 4 times from subsets
#### for hyperparameter tuning, need to consult with Daniel

In [379]:
#use kfold validation to train the model
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_absolute_error
import time

# Define the number of folds for cross-validation
n_splits = 5

#starttime for model run
start_time = time.time()

# Define the Gaussian Process model
kernel = RBF()
# kernel = ConstantKernel(constant_value = 3, constant_value_bounds=(1e-5, 1e5)) \
#               * RBF(length_scale=1, length_scale_bounds=(1e-5, 1e5))
gp_model_cv = GaussianProcessRegressor(kernel=kernel, random_state=42, n_restarts_optimizer= 4)

# Perform k-fold cross-validation and calculate MAE for each fold
mae_scores = cross_val_score(gp_model_cv, X_train, y_train, cv = n_splits, scoring='neg_mean_absolute_error')

print(mae_scores)
#endtime for model run
end_time = time.time()



/glade/u/home/sbhattarai/.conda/envs/meds-py/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:424: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/glade/u/home/sbhattarai/.conda/envs/meds-py/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:424: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/glade/u/home/sbhattarai/.conda/envs/meds-py/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:424: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/glade/

[-21.14571822 -29.31369955 -24.08539209 -30.16875394 -91.30085176]


/glade/u/home/sbhattarai/.conda/envs/meds-py/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:424: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


In [380]:
# I don't understand why model only predicting zeros, tried all possible approaches
gp_model_cv.predict(X_test)

array([0., 0., 0., ..., 0., 0., 0.])